In [1]:
import pandas as pd
from constants import DATA_PATH, EOS_FILE, SENTINEL_FILE

sentinel = pd.read_csv(DATA_PATH / SENTINEL_FILE)
eos = pd.read_csv(DATA_PATH / EOS_FILE)

sentinel = sentinel[sentinel['SM1 (%)'] != 50]
eos = eos[eos['SM1 (%)'] != 50]
from constants import X_cols_eos, X_cols_sentinel, y_col

X_sentinel = sentinel[X_cols_sentinel].values
X_eos = eos[X_cols_eos].values

y_sentinel = sentinel[y_col].values
y_eos = eos[y_col].values

In [2]:
import tensorflow as tf

models = {
#     "16, 1": tf.keras.Sequential([
#     # Input layer
#     tf.keras.Input(shape=(2, )),
#     tf.keras.layers.Dense(16, activation='relu'),
#     tf.keras.layers.Dense(1)
# ]),
#     "8, 1": tf.keras.Sequential([
#     # Input layer
#     tf.keras.Input(shape=(2, )),
#     tf.keras.layers.Dense(8, activation='relu'),
#     tf.keras.layers.Dense(1)
# ]),
#     "2, 1": tf.keras.Sequential([
#     # Input layer
#     tf.keras.Input(shape=(2, )),
#     tf.keras.layers.Dense(2, activation='relu'),
#     tf.keras.layers.Dense(1)
# ]),
#     "4, 1": tf.keras.Sequential([
#     # Input layer
#     tf.keras.Input(shape=(2, )),
#     tf.keras.layers.Dense(4, activation='relu'),
#     tf.keras.layers.Dense(1)
# ]),
#     "8, 4, 2, 1": tf.keras.Sequential([
#     # Input layer
#     tf.keras.Input(shape=(2, )),
#     tf.keras.layers.Dense(8, activation='relu'),
#     tf.keras.layers.Dense(4, activation='relu'),
#     tf.keras.layers.Dense(2, activation='relu'),
#     tf.keras.layers.Dense(1)
# ]),
    "16, Dropout, 8, Dropout": tf.keras.Sequential([
    # Input layer
    tf.keras.Input(shape=(2, )),
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dropout(0.09),
    tf.keras.layers.Dense(8, activation='relu'),
    tf.keras.layers.Dropout(0.09),
    tf.keras.layers.Dense(1)
]),
    "16, Dropout": tf.keras.Sequential([
    # Input layer
    tf.keras.Input(shape=(2, )),
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dropout(0.1),
    tf.keras.layers.Dense(1)
])
}

2025-11-20 05:39:20.838147: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
from model_experiments import ConformalizedQuantileExperiment

results_eos = {}

tf.keras.backend.clear_session()

ae = ConformalizedQuantileExperiment(X_eos, y_eos, satellite='EOS-04')
results = ae.run_experiment(models["16, Dropout, 8, Dropout"])

results


--- Running SVM Split Conformal (SVR) ---
  > Split Conformal Calibration constant (Q): 21.8484

--- Running Linear CQR (QuantileRegressor) ---
  > CQR Calibration constant (Q): -1.5667

--- Running ANN CQR ---


Epochs:  44%|████▍     | 44/100 [00:10<00:12,  4.39epoch/s, loss=0.4375, val_loss=0.4350]

7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step


7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
  > CQR Calibration constant (Q): -1.3111
Evaluating SVM_Split_Conformal...
Evaluating Linear_CQR...
Evaluating ANN_CQR...

All Conformal Experiments Completed. Results saved to /home/kshipra/work/coding/major/ml experiments/experiments/classification_new_data/output/conformal_results/EOS-04_conformal_metrics.json


{'SVM_Split_Conformal': {'PICP': 0.9365853658536586,
  'MPIW': 43.69683030039197},
 'Linear_CQR': {'PICP': 0.9073170731707317, 'MPIW': 40.33333333333331},
 'ANN_CQR': {'PICP': 0.9219512195121952, 'MPIW': 39.011204450498724}}

In [4]:
from model_experiments import ConformalizedQuantileExperiment

results_eos = {}

tf.keras.backend.clear_session()

ae = ConformalizedQuantileExperiment(X_sentinel, y_sentinel, satellite='Sentinel-1')
results = ae.run_experiment(models["16, Dropout, 8, Dropout"])

results


--- Running SVM Split Conformal (SVR) ---
  > Split Conformal Calibration constant (Q): 19.9989

--- Running Linear CQR (QuantileRegressor) ---
  > CQR Calibration constant (Q): -0.4667

--- Running ANN CQR ---


Epochs:  18%|█▊        | 18/100 [00:04<00:20,  4.04epoch/s, loss=0.4683, val_loss=0.4767]


5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/5 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/stepWARNING:tensorflow:5 out of the last 25 calls to <function TensorFlowTrainer.make_predict_function.<locals>.one_step_on_data_distributed at 0x7f71c84d3880> triggered tf.function retracing. Tracing is expensive and the excessive number of tracings could be due to (1) creating @tf.function repeatedly in a loop, (2) passing tensors with different shapes, (3) passing Python objects instead of tensors. For (1), please define your @tf.function outside of the loop. For (2), @tf.function has reduce_retracing=True option that can avoid unnecessary retracing. For (3), please refer to https://www.tensorflow.org/guide/function#controlling_retracing and https://www.tensorflow.org/api_docs/python/tf/function for  more details.
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step 
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
  > CQR Calibration constant (Q): -0.0329
Evaluating SVM_Split_Conformal...
Evaluating L

{'SVM_Split_Conformal': {'PICP': 0.9150326797385621,
  'MPIW': 39.99782968459487},
 'Linear_CQR': {'PICP': 0.9477124183006536, 'MPIW': 42.23333333333333},
 'ANN_CQR': {'PICP': 0.9411764705882353, 'MPIW': 41.867046133053854}}